In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile8 = pd.read_pickle("BBB_Train_Decile8.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile8.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile8, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 17s 5ms/step - loss: 0.3119 - acc: 0.8937 - val_loss: 0.2842 - val_acc: 0.8990
Epoch 59/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3060 - acc: 0.8915 - val_loss: 0.2832 - val_acc: 0.8990
Epoch 60/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3016 - acc: 0.8928 - val_loss: 0.2832 - val_acc: 0.8990
Epoch 61/75
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3054 - acc: 0.8860 - val_loss: 0.2826 - val_acc: 0.9003
Epoch 62/75
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2969 - acc: 0.8915 - val_loss: 0.2817 - val_acc: 0.9016
Epoch 63/75
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3021 - acc: 0.8889 - val_loss: 0.2831 - val_acc: 0.9003
Epoch 64/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3041 - acc: 0.8947 - val_loss: 0.2816 - val_acc: 0.8990
Epoch 65/75
3087/3087 [==============================] - 16s

3087/3087 [==============================] - 16s 5ms/step - loss: 0.2799 - acc: 0.8954 - val_loss: 0.2745 - val_acc: 0.9016
Epoch 42/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2753 - acc: 0.8957 - val_loss: 0.2744 - val_acc: 0.9016
Epoch 43/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2787 - acc: 0.8957 - val_loss: 0.2746 - val_acc: 0.9016
Epoch 44/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2816 - acc: 0.8973 - val_loss: 0.2746 - val_acc: 0.9016
Epoch 45/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2815 - acc: 0.8931 - val_loss: 0.2746 - val_acc: 0.9016
Epoch 46/75
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2767 - acc: 0.8970 - val_loss: 0.2745 - val_acc: 0.9016
Epoch 47/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2850 - acc: 0.8918 - val_loss: 0.2745 - val_acc: 0.9003
Epoch 48/75
3087/3087 [==============================] - 15s

3087/3087 [==============================] - 15s 5ms/step - loss: 0.2854 - acc: 0.8912 - val_loss: 0.2763 - val_acc: 0.8977
Epoch 34/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2806 - acc: 0.8934 - val_loss: 0.2757 - val_acc: 0.8977
Epoch 35/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2837 - acc: 0.8892 - val_loss: 0.2755 - val_acc: 0.8977
Epoch 36/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2801 - acc: 0.8918 - val_loss: 0.2749 - val_acc: 0.8990
Epoch 37/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2787 - acc: 0.8960 - val_loss: 0.2749 - val_acc: 0.8990

Epoch 00037: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 38/100
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2826 - acc: 0.8954 - val_loss: 0.2752 - val_acc: 0.8977
Epoch 39/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2833 - acc: 0.8925 - val_loss: 0.2754 - val_a

3087/3087 [==============================] - 15s 5ms/step - loss: 0.2835 - acc: 0.8912 - val_loss: 0.2751 - val_acc: 0.9016
Epoch 93/100
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2773 - acc: 0.8941 - val_loss: 0.2752 - val_acc: 0.9003
Epoch 94/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2823 - acc: 0.8937 - val_loss: 0.2751 - val_acc: 0.9003
Epoch 95/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2859 - acc: 0.8905 - val_loss: 0.2752 - val_acc: 0.9003
Epoch 96/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2856 - acc: 0.8934 - val_loss: 0.2751 - val_acc: 0.8977
Epoch 97/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2880 - acc: 0.8921 - val_loss: 0.2753 - val_acc: 0.8990
Epoch 98/100
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2875 - acc: 0.8928 - val_loss: 0.2754 - val_acc: 0.8977
Epoch 99/100
3087/3087 [==============================

...

3087/3087 [==============================] - 16s 5ms/step - loss: 0.2932 - acc: 0.8895 - val_loss: 0.2757 - val_acc: 0.9003
Epoch 16/50
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2998 - acc: 0.8931 - val_loss: 0.2796 - val_acc: 0.8925
Epoch 17/50
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2948 - acc: 0.8912 - val_loss: 0.2767 - val_acc: 0.8990

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 18/50
3087/3087 [==============================] - 17s 6ms/step - loss: 0.2924 - acc: 0.8925 - val_loss: 0.2744 - val_acc: 0.9028
Epoch 19/50
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2881 - acc: 0.8912 - val_loss: 0.2746 - val_acc: 0.9028
Epoch 20/50
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2808 - acc: 0.8908 - val_loss: 0.2749 - val_acc: 0.9016
Epoch 21/50
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2913 - acc: 0.8879 - val_loss: 0.2

Epoch 23/125
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3683 - acc: 0.8792 - val_loss: 0.3342 - val_acc: 0.8938
Epoch 24/125
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3668 - acc: 0.8811 - val_loss: 0.3325 - val_acc: 0.8990
Epoch 25/125
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3605 - acc: 0.8818 - val_loss: 0.3289 - val_acc: 0.8964
Epoch 26/125
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3535 - acc: 0.8834 - val_loss: 0.3273 - val_acc: 0.8951
Epoch 27/125
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3303 - acc: 0.8882 - val_loss: 0.3036 - val_acc: 0.8977
Epoch 45/125
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3253 - acc: 0.8886 - val_loss: 0.3011 - val_acc: 0.9003
Epoch 46/125
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3300 - acc: 0.8856 - val_loss: 0.3023 - val_acc: 0.8964
Epoch 47/125
3087/3087 [=================

3087/3087 [==============================] - 11s 3ms/step - loss: 0.3001 - acc: 0.8934 - val_loss: 0.2868 - val_acc: 0.9003
Epoch 115/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3022 - acc: 0.8947 - val_loss: 0.2860 - val_acc: 0.9003
Epoch 116/125
3087/3087 [==============================] - 11s 3ms/step - loss: 0.2998 - acc: 0.8912 - val_loss: 0.2863 - val_acc: 0.9003
Epoch 117/125
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3119 - acc: 0.8905 - val_loss: 0.2864 - val_acc: 0.8990
Epoch 118/125
3087/3087 [==============================] - 11s 3ms/step - loss: 0.3079 - acc: 0.8915 - val_loss: 0.2867 - val_acc: 0.8990
Epoch 119/125
3087/3087 [==============================] - 11s 3ms/step - loss: 0.2992 - acc: 0.8950 - val_loss: 0.2862 - val_acc: 0.9003
Epoch 120/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2988 - acc: 0.8941 - val_loss: 0.2864 - val_acc: 0.8990
Epoch 121/125
3087/3087 [=======================

Epoch 76/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2829 - acc: 0.8912 - val_loss: 0.2769 - val_acc: 0.8951
Epoch 77/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2838 - acc: 0.8934 - val_loss: 0.2771 - val_acc: 0.8951
Epoch 78/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2827 - acc: 0.8918 - val_loss: 0.2769 - val_acc: 0.8951
Epoch 79/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2808 - acc: 0.8889 - val_loss: 0.2769 - val_acc: 0.8964
Epoch 80/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2786 - acc: 0.8934 - val_loss: 0.2770 - val_acc: 0.8964
Epoch 81/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2792 - acc: 0.8947 - val_loss: 0.2769 - val_acc: 0.8951
Epoch 82/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2848 - acc: 0.8886 - val_loss: 0.2769 - val_acc: 0.8951
Epoch 83/100
3087/3087 [========================

Epoch 36/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2849 - acc: 0.8876 - val_loss: 0.2749 - val_acc: 0.8977
Epoch 37/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2978 - acc: 0.8908 - val_loss: 0.2762 - val_acc: 0.9016
Epoch 38/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2846 - acc: 0.8957 - val_loss: 0.2751 - val_acc: 0.8977

Epoch 00038: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 39/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2917 - acc: 0.8925 - val_loss: 0.2746 - val_acc: 0.8990
Epoch 40/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2762 - acc: 0.8980 - val_loss: 0.2747 - val_acc: 0.8977
Epoch 41/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2883 - acc: 0.8937 - val_loss: 0.2745 - val_acc: 0.9003
Epoch 42/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2882 - acc: 0.8950 - val_loss

In [ ]:
0.8297